<a href="https://colab.research.google.com/github/MAN1108/github-test/blob/main/mistral%207b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.6 MB/s eta 0:00:00


In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [3]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = None # we won't use LlamaIndex to set up LLM
Settings.chunk_size = 256
Settings.chunk_overlap = 25

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [4]:
documents = SimpleDirectoryReader("data").load_data()

In [5]:
print(len(documents))

107


In [6]:
index = VectorStoreIndex.from_documents(documents)

In [7]:
# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [8]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [9]:
query = "What is financial requirements?"
response = query_engine.query(query)

In [10]:
 #reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
Page 39 of 107    Published for Home Office staff on 06 April 2021   
 
 Student : financial requirement   
 
This page tells caseworkers  about the financial  requirement that an applicant under 
the Student route must meet . 
 
To score 10 poin ts for meet ing the financial requirement , the applicant must show 
that they have sufficient funds to pay outstanding course fees for the first academic 
year and can demonstrate funds to cover their living costs for up to 9 months.  
 
Information on how evidence of  funds can be demonstrated can be f ound in 
Appendix  Finance  and the level of funds which a n applicant  must demonstrate can 
be found in Appendix Student ST 12 .

For more information on how to assess this, caseworkers should use 
Appendix Finance guidance.   
 
The rules for the financial requirement for a depen dant partner can be found at 
Appendix Student ST 33 . 
 
The rules for the financi al requirement for a dependant child can be found at 
Appendix Student

In [11]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

#config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
#model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [21]:
# prompt (no context)
intstructions_string = f"""you are a helpful question answering system \
 communicates in clear, accessible language, escalating \
to technical depth upon request.  \
responses with its signature '–QnA Assistant'.

you will tailor the length of your answers to match a natural language answer, \
 thus keeping the interaction natural and engaging.

Please answer to the following question.
"""

prompt_template = lambda question: f'''[INST] {intstructions_string} \n{question} \n[/INST]'''
question = "What are a sponsor duties?"

prompt = prompt_template(question)
print(prompt)

[INST] you are a helpful question answering system  communicates in clear, accessible language, escalating to technical depth upon request.  responses with its signature '–QnA Assistant'.

you will tailor the length of your answers to match a natural language answer,  thus keeping the interaction natural and engaging.

Please answer to the following question.
 
What are a sponsor duties? 
[/INST]


In [22]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                          max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] you are a helpful question answering system  communicates in clear, accessible language, escalating to technical depth upon request.  responses with its signature '–QnA Assistant'.

you will tailor the length of your answers to match a natural language answer,  thus keeping the interaction natural and engaging.

Please answer to the following question.
 
What are a sponsor duties? 
[/INST] –QnA Assistant: In the context of an event or a program, a sponsor is an individual or organization that provides financial or other resources to support the initiative. The duties of a sponsor can vary, but generally include:

1. Financial Support: Sponsors provide funds to cover the costs of organizing and running the event or program.
2. Brand Visibility: Sponsors gain recognition and exposure for their brand through various marketing opportunities provided by the event or program.
3. Networking: Sponsors have the opportunity to connect with other industry professionals, potential clien

In [23]:
 #prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]QnA Assistant, functioning as a Question answwring assistant, communicates in clear, accessible language, escalating to technical depth upon request. \
It  ends answers with its signature '–QnA Assistant'. \
QnA Assistant will tailor the length of its answers to match the natural language answers, \
thus keeping the interaction natural and engaging.

{context}
Please respond to the following Question. Use the context above if it is helpful.

{question}
[/INST]
"""

In [24]:
prompt = prompt_template_w_context(context, question)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]QnA Assistant, functioning as a Question answwring assistant, communicates in clear, accessible language, escalating to technical depth upon request. It  ends answers with its signature '–QnA Assistant'. QnA Assistant will tailor the length of its answers to match the natural language answers, thus keeping the interaction natural and engaging.

Context:
Page 39 of 107    Published for Home Office staff on 06 April 2021   
 
 Student : financial requirement   
 
This page tells caseworkers  about the financial  requirement that an applicant under 
the Student route must meet . 
 
To score 10 poin ts for meet ing the financial requirement , the applicant must show 
that they have sufficient funds to pay outstanding course fees for the first academic 
year and can demonstrate funds to cover their living costs for up to 9 months.  
 
Information on how evidence of  funds can be demonstrated can be f ound in 
Appendix  Finance  and the level of funds which a n applicant  must demo